In [1]:
import tqdm
import time
import numpy as np
import pandas as pd

from sklearn.utils import check_random_state
from lightgbm import LGBMRanker
from sharp import ShaRP
from sharp.utils import scores_to_ordering
from xai_ranking.preprocessing import preprocess_higher_education_data
from xai_ranking.scorers import higher_education_score
from mlresearch.utils import check_random_states

from xai_ranking.preprocessing import (
    preprocess_atp_data,
    preprocess_csrank_data,
    preprocess_higher_education_data,
    preprocess_movers_data,
)
from xai_ranking.datasets import (
    fetch_atp_data,
    fetch_csrank_data,
    fetch_higher_education_data,
    fetch_movers_data,
)
from xai_ranking.scorers import (
    atp_score,
    csrank_score,
    higher_education_score,
)
from xai_ranking.metrics import (
    explanation_sensitivity, outcome_sensitivity,
    bootstrapped_explanation_consistency, cross_method_explanation_consistency,
    cross_method_outcome_consistency
)

RNG_SEED = 42
N_RUNS = 5

In [2]:
# Set up ranker for the moving company dataset:
X, ranks, score = preprocess_movers_data(fetch_movers_data(test=False))
qids_train = X.index.value_counts().to_numpy()

model = LGBMRanker(
    objective="lambdarank", label_gain=list(range(max(ranks) + 1)), verbose=-1
)
model.fit(
    X=X,
    y=ranks,
    group=qids_train,
)

LGBMRanker(label_gain=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16,
                       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, ...],
           objective='lambdarank', verbose=-1)

In [3]:
random_states = check_random_states(RNG_SEED, N_RUNS)

datasets = [
    {
        "name": "ATP",
        "data": preprocess_atp_data(fetch_atp_data()),
        "scorer": atp_score,
        "n_observations": 86,
    },
    {
        "name": "CSRank",
        "data": preprocess_csrank_data(fetch_csrank_data()),
        "scorer": csrank_score,
        "n_observations": 100,
    },
    {
        "name": "Higher Education",
        "data": preprocess_higher_education_data(
            fetch_higher_education_data(year=2020)
        ),
        "scorer": higher_education_score,
        "n_observations": 100,
    },
    {
        "name": "Moving Company",
        "data": preprocess_movers_data(fetch_movers_data(test=True)),
        "scorer": model.predict,
        "n_observations": 100,
    },
]

approaches = ["rank", "score", "pairwise"]

default_kwargs = {
    "measure": "shapley",
    "sample_size": None,
    "coalition_size": None,
    "replace": True,
    "n_jobs": 1,
}
parameters_to_change = {
    "coalition_size": [i for i in range(2, 6)],
    "sample_size": [i for i in np.arange(.1, 1.1, .1)],
    "n_jobs": [i for i in range(1, 32, 2)],
}

In [4]:
def outcome_fidelity(contributions, target, avg_target, target_pairs=None):
    if target_pairs is None:
        avg_est_err = np.mean(np.abs(target - contributions.sum(axis=1) + avg_target))
    else:
        better_than = target > target_pairs
        est_better_than = contributions.sum(axis=1) > 0
        avg_est_err = (better_than == est_better_than).mean()
    return avg_est_err

In [5]:
# Super janky code... It would be a good exercise to refactor this

result_cols = (
    ["dataset", "n_observations", "approach", "parameter", "parameter_value", "avg_time"]
    + [f"time_{i}" for i in range(N_RUNS)]
    + [f"exp_cons_kendall_{i}" for i in range(N_RUNS)]
    + [f"exp_sens_kendall_{i}" for i in range(N_RUNS)]
    + [f"exp_cons_jaccard2_{i}" for i in range(N_RUNS)]
    + [f"exp_sens_jaccard2_{i}" for i in range(N_RUNS)]
    + [f"exp_cons_euclidean_{i}" for i in range(N_RUNS)]
    + [f"exp_sens_euclidean_{i}" for i in range(N_RUNS)]
    + [f"fidelity_{i}" for i in range(N_RUNS)]
)

result_df = []

for dataset in datasets:

    # Set up basic settings
    X = dataset["data"][0]
    scorer = dataset["scorer"]
    scores = scorer(dataset["data"][0])
    ranking = scores_to_ordering(scores)

    rng = check_random_state(RNG_SEED)
    sam_idx1 = rng.choice(
        np.indices((X.shape[0],)).squeeze(), size=dataset["n_observations"], replace=False
    )
    sam_idx2 = rng.choice(
        np.indices((X.shape[0],)).squeeze(), size=dataset["n_observations"], replace=False
    )

    for approach in approaches:
        print("----------------", dataset["name"], "|", approach, "----------------")

        times = []
        kendall_cons = []
        kendall_sens = []
        jaccard_cons = []
        jaccard_sens = []
        euclidean_cons = []
        euclidean_sens = []
        fidelity = []

        print("Exact computation")
        for i in tqdm.tqdm(range(N_RUNS)):
            start = time.time()
            if approach != "pairwise":
                baseline_sharp = ShaRP(
                    qoi=approach,
                    target_function=dataset["scorer"],
                    random_state=random_states[i],
                    **default_kwargs,
                )
                baseline_sharp.fit(X)
                baseline_contr = baseline_sharp.all(X.iloc[sam_idx1])
            else:
                baseline_sharp = ShaRP(
                    target_function=dataset["scorer"],
                    random_state=random_states[i],
                    **default_kwargs,
                )
                baseline_pairwise = []
                for idx1, idx2 in zip(sam_idx1, sam_idx2):
                    baseline_pairwise.append(baseline_sharp.pairwise(X.iloc[idx1], X.iloc[idx2]))
                baseline_pairwise = np.array(baseline_pairwise)
                
            end = time.time()

            baseline_contr = pd.DataFrame(baseline_contr, columns=X.columns, index=X.iloc[sam_idx1].index)
            # Save metrics
            times.append(end - start)
            kendall_cons.append(0)
            kendall_sens.append(
                explanation_sensitivity(X.iloc[sam_idx1], baseline_contr, scores_to_ordering(scores[sam_idx1]), measure="kendall")
                if approach != "pairwise" else np.nan
            )
            jaccard_cons.append(0)
            jaccard_sens.append(
                explanation_sensitivity(X.iloc[sam_idx1], baseline_contr, scores_to_ordering(scores[sam_idx1]), measure="jaccard", n_features=2)
                if approach != "pairwise" else np.nan
            )
            euclidean_cons.append(0)
            euclidean_sens.append(
                explanation_sensitivity(X.iloc[sam_idx1], baseline_contr, scores_to_ordering(scores[sam_idx1]), measure="euclidean")
                if approach != "pairwise" else np.nan
            )

            if approach != "pairwise":
                target = scores if approach == "score" else ranking
                avg_target = target.mean()
                res_ = outcome_fidelity(baseline_contr, target[sam_idx1], avg_target)
            else:
                res_ = outcome_fidelity(baseline_pairwise, target[sam_idx1], avg_target, target_pairs=target[sam_idx2])

            fidelity.append(res_)

        exact_results_row = (
            [
                dataset["name"], 
                dataset["n_observations"], 
                approach, 
                np.nan, 
                np.nan, 
                np.mean(times)
            ] + times + kendall_cons + kendall_sens + jaccard_cons +
            jaccard_sens + euclidean_cons + euclidean_sens + fidelity
        )
        result_df.append(exact_results_row)
        print("Finished computing exact results")
        ############################################################################################

        for parameter, parameter_values in parameters_to_change.items():
            print(f"Alternating parameter: {parameter}")
            default_value = default_kwargs[parameter] if parameter in default_kwargs else None
            for parameter_value in tqdm.tqdm(parameter_values):

                if parameter == "sample_size":
                    parameter_value = int(parameter_value*X.shape[0])
                    
                default_kwargs[parameter] = parameter_value

                times = []
                kendall_cons = []
                kendall_sens = []
                jaccard_cons = []
                jaccard_sens = []
                euclidean_cons = []
                euclidean_sens = []
                fidelity = []

                print(f"Parameter {parameter}, value {parameter_value}")
                for i in tqdm.tqdm(range(N_RUNS)):
                    start = time.time()
                    if approach != "pairwise":
                        sharp = ShaRP(
                            qoi=approach,
                            target_function=dataset["scorer"],
                            random_state=random_states[i],
                            **default_kwargs,
                        )
                        sharp.fit(X)
                        contr = sharp.all(X.iloc[sam_idx1])
                    else:
                        sharp = ShaRP(
                            target_function=dataset["scorer"],
                            random_state=random_states[i],
                            **default_kwargs,
                        )
                        pairwise = []
                        for idx1, idx2 in zip(sam_idx1, sam_idx2):
                            pairwise.append(sharp.pairwise(X.iloc[idx1], X.iloc[idx2]))
                        pairwise = np.array(pairwise)

                    end = time.time()

                    contr = pd.DataFrame(contr, columns=X.columns, index=X.iloc[sam_idx1].index)

                    # Save metrics
                    times.append(end - start)
                    kendall_cons.append(
                        cross_method_explanation_consistency(contr, baseline_contr, measure="kendall")
                    )
                    kendall_sens.append(
                        explanation_sensitivity(X.iloc[sam_idx1], contr, scores_to_ordering(scores[sam_idx1]), measure="kendall")
                        if approach != "pairwise" else np.nan
                    )
                    jaccard_cons.append(
                        cross_method_explanation_consistency(contr, baseline_contr, measure="jaccard", n_features=2)
                    )
                    jaccard_sens.append(
                        explanation_sensitivity(X.iloc[sam_idx1], contr, scores_to_ordering(scores[sam_idx1]), measure="jaccard", n_features=2)
                        if approach != "pairwise" else np.nan
                    )
                    euclidean_cons.append(
                        cross_method_explanation_consistency(contr, baseline_contr, measure="euclidean")
                    )
                    euclidean_sens.append(
                        explanation_sensitivity(X.iloc[sam_idx1], contr, scores_to_ordering(scores[sam_idx1]), measure="euclidean")
                        if approach != "pairwise" else np.nan
                    )
                    if approach != "pairwise":
                        target = scores if approach == "score" else ranking
                        avg_target = target.mean()
                        res_ = outcome_fidelity(contr, target[sam_idx1], avg_target)
                    else:
                        res_ = outcome_fidelity(pairwise, target[sam_idx1], avg_target, target_pairs=target[sam_idx2])

                    fidelity.append(res_)

                results_row = (
                    [
                        dataset["name"], 
                        dataset["n_observations"], 
                        approach, 
                        parameter, 
                        parameter_value, 
                        np.mean(times)
                    ] + times + kendall_cons + kendall_sens + jaccard_cons +
                    jaccard_sens + euclidean_cons + euclidean_sens + fidelity
                )
                result_df.append(results_row)
                print(f"Stored results for {parameter} | {parameter_value}")

            default_kwargs[parameter] = default_value


results = pd.DataFrame(result_df, columns=result_cols)
results.to_csv("results/time-experiment-" + dataset["name"] + ".csv")

---------------- ATP | rank ----------------
Exact computation


  0%|          | 0/5 [00:00<?, ?it/s]

/tmp/ipykernel_5091/4255801183.py:75: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  explanation_sensitivity(X.iloc[sam_idx1], baseline_contr, scores_to_ordering(scores[sam_idx1]), measure="kendall")
/tmp/ipykernel_5091/4255801183.py:80: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  explanation_sensitivity(X.iloc[sam_idx1], baseline_contr, scores_to_ordering(scores[sam_idx1]), measure="jaccard", n_features=2)
/tmp/ipykernel_5091/4255801183.py:85: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame beha

Finished computing exact results
Alternating parameter: coalition_size


  0%|          | 0/4 [00:00<?, ?it/s]

...

Stored results for sample_size | 8
Parameter sample_size, value 17


/tmp/ipykernel_5091/4255801183.py:165: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  explanation_sensitivity(X.iloc[sam_idx1], contr, scores_to_ordering(scores[sam_idx1]), measure="kendall")
/tmp/ipykernel_5091/4255801183.py:172: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  explanation_sensitivity(X.iloc[sam_idx1], contr, scores_to_ordering(scores[sam_idx1]), measure="jaccard", n_features=2)
/tmp/ipykernel_5091/4255801183.py:179: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To acces

Stored results for sample_size | 17
Parameter sample_size, value 25


/tmp/ipykernel_5091/4255801183.py:165: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  explanation_sensitivity(X.iloc[sam_idx1], contr, scores_to_ordering(scores[sam_idx1]), measure="kendall")
/tmp/ipykernel_5091/4255801183.py:172: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  explanation_sensitivity(X.iloc[sam_idx1], contr, scores_to_ordering(scores[sam_idx1]), measure="jaccard", n_features=2)
/tmp/ipykernel_5091/4255801183.py:179: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To acces

KeyboardInterrupt: 

In [ ]:
results = pd.DataFrame(result_df, columns=result_cols)
results

,dataset,n_observations,approach,parameter,parameter_value,avg_time,time_0,exp_cons_kendall_0,exp_sens_kendall_0,exp_cons_jaccard2_0,exp_sens_jaccard2_0,exp_cons_euclidean_0,exp_sens_euclidean_0,fidelity_0
0,ATP,86,rank,NaN,NaN,44.691251,44.691251,0,"(0.1803100775193798, 0.031855637608718734)",0,"(0.3542635658914728, 0.02125817242206513)",0,"(0.5055116701207965, 0.020296555616382925)",87.028011
1,ATP,86,rank,coalition_size,2.0,44.790097,44.790097,"(5.551115123125783e-17, 0.0)","(0.1803100775193798, 0.031855637608718734)","(0.0, 0.0)","(0.3542635658914728, 0.02125817242206513)","(0.0, 0.0)","(0.5055116701207965, 0.020296555616382925)",87.028011
2,ATP,86,rank,coalition_size,3.0,44.426559,44.426559,"(5.551115123125783e-17, 0.0)","(0.1803100775193798, 0.031855637608718734)","(0.0, 0.0)","(0.3542635658914728, 0.02125817242206513)","(0.0, 0.0)","(0.5055116701207965, 0.020296555616382925)",87.028011
3,ATP,86,rank,coalition_size,4.0,44.679947,44.679947,"(5.551115123125783e-17, 0.0)","(0.1803100775193798, 0.031855637608718734)","(0.0, 0.0)","(0.3542635658914728, 0.02125817242206513)","(0.0, 0.0)","(0.5055116701207965, 0.020296555616382925)",87.028011
4,ATP,86,rank,coalition_size,5.0,45.065957,45.065957,"(5.551115123125783e-17, 0.0)","(0.1803100775193798, 0.031855637608718734)","(0.0, 0.0)","(0.3542635658914728, 0.02125817242206513)","(0.0, 0.0)","(0.5055116701207965, 0.020296555616382925)",87.028011
5,ATP,86,rank,sample_size,86.0,48.558038,48.558038,"(5.551115123125783e-17, 0.0)","(0.1803100775193798, 0.031855637608718734)","(0.0, 0.0)","(0.3542635658914728, 0.02125817242206513)","(0.0, 0.0)","(0.5055116701207965, 0.020296555616382925)",87.028011
6,ATP,86,rank,sample_size,64.0,36.086898,36.086898,"(0.021705426356589196, 0.004956618543110581)","(0.1851162790697674, 0.03211542386013017)","(0.06976744186046512, 0.02200531964311404)","(0.34573643410852717, 0.019945591153493644)","(0.027162132754206255, 0.0021569182674204194)","(0.5075600883927767, 0.020171649415067622)",86.996630
7,ATP,86,rank,sample_size,43.0,22.819893,22.819893,"(0.021705426356589196, 0.004833861451427646)","(0.15426356589147286, 0.03307637805954728)","(0.054263565891472874, 0.01965728931934371)","(0.337984496124031, 0.02057670708728971)","(0.02874875744862143, 0.002259680406451926)","(0.5072292743682366, 0.020072409331519058)",86.840828
8,ATP,86,rank,sample_size,21.0,12.274754,12.274754,"(0.027906976744186098, 0.0054224921210565356)","(0.16945736434108527, 0.03167109551568655)","(0.046511627906976744, 0.018313935603678094)","(0.34767441860465115, 0.020601354558600876)","(0.035685857050441416, 0.0027115618303575123)","(0.5076424683399984, 0.0199311242926804)",86.799059
9,ATP,86,rank,n_jobs,1.0,43.450870,43.450870,"(5.551115123125783e-17, 0.0)","(0.1803100775193798, 0.031855637608718734)","(0.0, 0.0)","(0.3542635658914728, 0.02125817242206513)","(0.0, 0.0)","(0.5055116701207965, 0.020296555616382925)",87.028011
